In [1]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

device

device(type='cuda')

In [4]:
#!pip install pylint gymnasium stable_baselines3

# Voltar daqui

In [5]:
import gymnasium as gym
from gymnasium import spaces
import numpy as np
import pandas as pd
import tempfile
import subprocess
import shutil
from transformers import pipeline
import os
from stable_baselines3 import PPO
from stable_baselines3.common.env_checker import check_env
from stable_baselines3.common.vec_env import DummyVecEnv
import logging
from stable_baselines3.common.callbacks import BaseCallback
from stable_baselines3.common.logger import configure
import matplotlib.pyplot as plt

# Configurando ambiente

In [6]:
# Configurar logging
logging.basicConfig(filename='agentes_conversa.log', level=logging.INFO, format='%(asctime)s - %(message)s')

class DataAnalysisEnv(gym.Env):
    """
    Ambiente personalizado para agentes cooperativos de análise de dados.
    """

    def __init__(self, data_path):
        super(DataAnalysisEnv, self).__init__()

        # Carrega o dataset Walmart
        self.data = pd.read_csv(data_path)

        # Definição dos espaços de ação para os dois agentes (combinados em um Dict)
        self.action_space = spaces.MultiDiscrete([4, 5])  # Codificador tem 4 ações, Revisor tem 5 ações

        # Definindo um espaço para as observações numéricas
        self.max_codigo_length = 1000
        self.max_relatorio_length = 1500

        total_length = self.max_codigo_length + self.max_relatorio_length + 1  # Código, Relatório e o score

        # Espaço de observação agora é um único Box contínuo
        self.observation_space = spaces.Box(
            low=0,
            high=1,  # Normalizado para ficar entre 0 e 1
            shape=(total_length,),
            dtype=np.float32
        )

        # Inicialização das variáveis de estado
        self.state = {
            "codigo": np.zeros(self.max_codigo_length, dtype=np.int32),
            "relatorio": np.zeros(self.max_relatorio_length, dtype=np.int32),
            "score_relatorio": np.array([0.0], dtype=np.float32)
        }

        self.threshold_score = 135  # 90% de 150 pontos

        self.temp_dir = tempfile.mkdtemp()
        self.linter = 'pylint'

        # Modelos LLM para Codificador e Revisor
        self.codificador_llm = pipeline('text-generation', model='gpt2')
        self.revisor_llm = pipeline('text-generation', model='gpt2')

    def reset(self, seed=None, options=None):
        # Configura a semente para garantir repetibilidade (se aplicável)
        super().reset(seed=seed)
        if seed is not None:
            np.random.seed(seed)

        # Redefine o estado
        self.state = {
            "codigo": np.zeros(self.max_codigo_length, dtype=np.int32),
            "relatorio": np.zeros(self.max_relatorio_length, dtype=np.int32),
            "score_relatorio": np.array([0.0], dtype=np.float32)
        }

        # Retorna a observação como um vetor contínuo
        observation = np.concatenate([
            self.state['codigo'] / 65535,  # Normalizando valores Unicode entre 0 e 1
            self.state['relatorio'] / 65535,
            self.state['score_relatorio'] / 150
        ]).astype(np.float32)

        return observation, {}

    def step(self, actions):
        done = False
        info = {}

        # Ações dos agentes
        action_codificador = actions[0]
        action_revisor = actions[1]

        # Codificador gera código
        codigo_gerado = self.execute_action_codificador(action_codificador)
        self.state['codigo'][:len(codigo_gerado)] = [ord(c) for c in codigo_gerado]  # Usa ord() para valores Unicode

        # Log e print da ação do codificador
        logging.info(f"Codificador ação: {action_codificador} -> Código gerado:\n{codigo_gerado}")
        print(f"Codificador ação: {action_codificador} -> Código gerado:\n{codigo_gerado}")

        # Executa e avalia o código
        exec_result, exec_error = self.avaliar_codigo(codigo_gerado)
        linter_output = self.rodar_linter(codigo_gerado)

        codigo_esta_correto = exec_error is None

        # Revisor gera feedback
        feedback = self.execute_action_revisor(action_revisor, exec_error, linter_output)
        self.state['relatorio'][:len(feedback)] = [ord(c) for c in feedback]  # Usa ord() para valores Unicode

        # Log e print da ação do revisor
        logging.info(f"Revisor ação: {action_revisor} -> Feedback gerado:\n{feedback}")
        print(f"Revisor ação: {action_revisor} -> Feedback gerado:\n{feedback}")

        # Calcula a pontuação do relatório
        self.state['score_relatorio'] = self.calcular_score_relatorio(feedback)

        # Calcular recompensa
        reward_codificador = 1.0 if codigo_esta_correto else -1.0
        reward_revisor = self.state['score_relatorio'][0] / 150
        combined_reward = (reward_codificador + reward_revisor) / 2  # Média das recompensas

        # Log e print das recompensas
        logging.info(f"Recompensa Codificador: {reward_codificador}, Recompensa Revisor: {reward_revisor}, Recompensa Combinada: {combined_reward}")
        print(f"Recompensa Codificador: {reward_codificador}, Recompensa Revisor: {reward_revisor}, Recompensa Combinada: {combined_reward}")

        # Verificar se o episódio terminou
        if self.state['score_relatorio'] >= self.threshold_score:
            done = True

        # Concatenar código, relatório e score em uma única observação
        observation = np.concatenate([
            self.state['codigo'] / 65535,  # Normalizando valores Unicode entre 0 e 1
            self.state['relatorio'] / 65535,
            self.state['score_relatorio'] / 150  # Normalizando o score entre 0 e 1
        ]).astype(np.float32)

        return observation, combined_reward, done, False, info  # Retornando o valor `False` para truncated

    def execute_action_codificador(self, action):
        actions_map = {
            0: "Processar os dados.",
            1: "Analisar os dados.",
            2: "Visualizar os resultados.",
            3: "Interpretar a análise."
        }

        action_text = actions_map.get(action, "")
        prompt = f"Escreva uma função Python para {action_text} usando a coluna de Date e Weekly_Sales do dataset Walmart.csv de vendas da Walmart para realizar uma análise temporal."

        # Gera o código, mas limita o número de tokens gerados
        generated = self.codificador_llm(prompt, max_new_tokens=50, num_return_sequences=1)
        codigo = generated[0]['generated_text']

        # Truncar o código gerado para o tamanho máximo permitido
        codigo = codigo[:1024]  # Truncando para 1024 caracteres

        return codigo

    def execute_action_revisor(self, action, exec_error, linter_output):
        actions_map = {
            0: "Análise estática do código.",
            1: "Executar o código.",
            2: "Propor refatorações.",
            3: "Aprovar o código.",
            4: "Melhorar o relatório."
        }

        action_text = actions_map.get(action, "")
        prompt = f"{action_text}\nCódigo:\n{''.join([chr(c) for c in self.state['codigo'] if c > 0])}\nErro:\n{exec_error}\nLinter:\n{linter_output}\n"

        # Gera o feedback, mas limita o número de tokens gerados
        generated = self.revisor_llm(prompt, max_new_tokens=100, num_return_sequences=1)
        feedback = generated[0]['generated_text']

        # Truncar o feedback para o tamanho máximo permitido
        feedback = feedback[:1024]  # Truncando para 1024 caracteres

        return feedback

    def avaliar_codigo(self, codigo):
        try:
            temp_code_path = os.path.join(self.temp_dir, 'temp_code.py')
            with open(temp_code_path, 'w') as f:
                f.write(codigo)

            result = subprocess.run(['python', temp_code_path], capture_output=True, text=True, timeout=10)

            if result.returncode == 0:
                return result.stdout, None
            else:
                return None, result.stderr

        except Exception as e:
            return None, str(e)

    def rodar_linter(self, codigo):
        try:
            temp_code_path = os.path.join(self.temp_dir, 'temp_code.py')
            with open(temp_code_path, 'w') as f:
                f.write(codigo)

            process = subprocess.run([self.linter, temp_code_path], capture_output=True, text=True, timeout=10)
            return process.stdout
        except Exception as e:
            return str(e)

    def calcular_score_relatorio(self, relatorio):
        score_total = min(len(relatorio.split()), 150)
        return np.array([float(score_total)], dtype=np.float32)

    def render(self):
        pass

    def close(self):
        shutil.rmtree(self.temp_dir)


# Treinando o modelo

In [ ]:
# Criar e verificar o ambiente com o arquivo CSV Walmart
data_path = 'Walmart.csv'
env = DataAnalysisEnv(data_path)
check_env(env)

# Vectorizando o ambiente para o treinamento com múltiplos episódios
vec_env = DummyVecEnv([lambda: env])

# Configurando o logger para registrar os logs
tmp_path = "./logs/"
new_logger = configure(tmp_path, ["stdout", "csv", "tensorboard"])

# Inicializando e treinando o modelo
model = PPO("MlpPolicy", vec_env, verbose=1, device=device)
model.set_logger(new_logger)

model.learn(total_timesteps=30000)

# Salvando o modelo treinado
model.save("ppo_walmart_agents")

# Testando o ambiente após o treinamento
obs, _ = env.reset()
for _ in range(100):
    action, _ = model.predict(obs)
    obs, reward, done, truncated, info = env.step(action)
    if done:
        obs, _ = env.reset()


# Para carregar o modelo mais tarde
# model = PPO.load("ppo_data_analysis")

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.
Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Codificador ação: 2 -> Código gerado:
Escreva uma função Python para Visualizar os resultados. usando a coluna de Date e Weekly_Sales do dataset Walmart.csv de vendas da Walmart para realizar uma análise temporal. Pida es não não. Pida e Realizar para otração. Viva de a otroçado os otração. Todo ácos hacer que ser puedes ser pued


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Revisor ação: 3 -> Feedback gerado:
Aprovar o código.
Código:
Escreva uma função Python para Visualizar os resultados. usando a coluna de Date e Weekly_Sales do dataset Walmart.csv de vendas da Walmart para realizar uma análise temporal. Pida es não não. Pida e Realizar para otração. Viva de a otroçado os otração. Todo ácos hacer que ser puedes ser pued
Erro:
  File "/tmp/tmpe0wq_po9/temp_code.py", line 1
    Escreva uma função Python para Visualizar os resultados. usando a coluna de Date e Weekly_Sales do dataset Walmart.csv de vendas da Walmart para realizar uma análise temporal. Pida es não não. Pida e Realizar para otração. Viva de a otroçado os otração. Todo ácos hacer que ser puedes ser pued
            ^^^
SyntaxError: invalid syntax

Linter:
************* Module temp_code
/tmp/tmpe0wq_po9/temp_code.py:1:9: E0001: Parsing failed: 'invalid syntax (temp_code, line 1)' (syntax-error)


Todo:

Treat "invalid syntax (output field) as "in_array".


Compose and execute the program (see

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


...

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Revisor ação: 2 -> Feedback gerado:
Propor refatorações.
Código:
Escreva uma função Python para Analisar os dados. usando a coluna de Date e Weekly_Sales do dataset Walmart.csv de vendas da Walmart para realizar uma análise temporal. en tanto no eso o nel viedo de todos baja e almanos í el mundo su ente o público. a lo que não que o a estando dos deja lo
Erro:
  File "/tmp/tmpe0wq_po9/temp_code.py", line 1
    Escreva uma função Python para Analisar os dados. usando a coluna de Date e Weekly_Sales do dataset Walmart.csv de vendas da Walmart para realizar uma análise temporal. en tanto no eso o nel viedo de todos baja e almanos í el mundo su ente o público. a lo que não que o a estando dos deja lo
            ^^^
SyntaxError: invalid syntax

Linter:
************* Module temp_code
/tmp/tmpe0wq_po9/temp_code.py:1:9: E0001: Parsing failed: 'invalid syntax (temp_code, line 1)' (syntax-error)


Analisar:

E0001: Cannot parse 'invalid syntax (temp_code, line 1)' with 'line 1' in module temp_c

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Revisor ação: 2 -> Feedback gerado:
Propor refatorações.
Código:
Escreva uma função Python para Processar os dados. usando a coluna de Date e Weekly_Sales do dataset Walmart.csv de vendas da Walmart para realizar uma análise temporal. e sintre un estado o deja para Realizar de Realize, e sintres e da ejercito e Sítica. da ejerciteção os ejé años de Sales
Erro:
  File "/tmp/tmpe0wq_po9/temp_code.py", line 1
    Escreva uma função Python para Processar os dados. usando a coluna de Date e Weekly_Sales do dataset Walmart.csv de vendas da Walmart para realizar uma análise temporal. e sintre un estado o deja para Realizar de Realize, e sintres e da ejercito e Sítica. da ejerciteção os ejé años de Sales
            ^^^
SyntaxError: invalid syntax

Linter:
************* Module temp_code
/tmp/tmpe0wq_po9/temp_code.py:1:9: E0001: Parsing failed: 'invalid syntax (temp_code, line 1)' (syntax-error)


<-- line 1 of file

===== (SyntaxError): invalid syntax (line 1)

ERROR: 'invalid syntax' is not s

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
